In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

torch.manual_seed(1)

In [2]:
# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('../Datasets/SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, kernel_size, hidden_size, drop_prob):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 1, kernel_size),
            nn.Flatten(),
            nn.Dropout(drop_prob),
            nn.Linear(input_size - kernel_size + 1, hidden_size[0]),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.Linear(hidden_size[1], hidden_size[2]),
            #nn.LogSoftmax(dim=1), # NLLLoss
            nn.Softmax(dim=1) # MSELoss
        )
        
    def forward(self, x):
        return self.model(x)

In [4]:
BEAT_TYPES_INDEX = {
    'N': 0,
    'L': 1,
    'R': 2,
    'A': 3,
    'V': 4,
    #'F': 5,
}

def formatDataX(data):
    temp = torch.tensor(data).float()
    return temp.view(temp.size(0), 1, temp.size(1))

def formatDataY(data):
    return torch.tensor(data).float()

def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    labels = df.iloc[:, -1].values
    Y = np.array([np.insert(np.zeros(5), BEAT_TYPES_INDEX[label], 1) for label in labels])

    return formatDataX(X), formatDataY(Y)


In [5]:
X, Y = preprocess(data)

In [68]:
torch.load("../Models/CurrentBest.pt")

NeuralNetwork(
  (model): Sequential(
    (0): Conv1d(1, 1, kernel_size=(20,), stride=(1,))
    (1): Flatten()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=181, out_features=64, bias=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=6, bias=True)
    (6): Softmax(dim=1)
  )
)

In [66]:
def test(net, X, Y, print_conf=True):
    resValues, resIndices = torch.max(net(X), 1)
    length = len(resIndices.numpy())
    testValues, testIndices = torch.max(Y, 1)
    results = np.array([np.insert(np.zeros(5), index, 1) for index in resIndices])
    
    conf_matrix = confusion_matrix(resIndices.numpy(), testIndices.numpy())

    correct = 0
    for i in range(len(conf_matrix)):
        correct += conf_matrix[i,i]
        
    print("Accuracy: {:.2%}".format(correct/length))
    if print_conf:
        print("Confusion Matrix:")
        print(conf_matrix)

In [67]:
test(trained_NN, X, Y)

Accuracy: 85.58%
Confusion Matrix:
[[72651  3219  2464   953  1840   502]
 [ 1032  3804    64   124   515    77]
 [  267   190  4669   112   197     7]
 [  186   453    30  1123   236     5]
 [  447   351     5   218  3561     5]
 [  139    52    18    14   769   206]]
